In [20]:
!pip install dotenv
!pip install langchain
!pip install python-dotenv
!pip install streamlit
!pip install tiktoken
!pip install faiss-cpu
!pip install protobuf
!pip install InstructorEmbedding
!pip install sentence-transformers

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=65fd59dbcdb8bb5f2dda25c7daf69529876e4eba43bad7201d12adf3a3bbcd46
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe8510

In [11]:
# from langchain.llms import GooglePalm
from langchain_community.chat_models import ChatGooglePalm

In [3]:
api_key = "GOOGLE_API_KEY"

In [12]:
# llm = GooglePalm(google_api_key=api_key, temperature=0.2)
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.2)
anime = llm.invoke("Get me a List of best short anime")
print(anime)

content="Here is a list of best short anime:\n\n1. **Death Parade** (12 episodes)\n\nDeath Parade is a psychological anime series that follows the arbiters Decim and Chiyuki as they judge the souls of the recently deceased in a bar called Quindecim. The series explores themes of life, death, and the afterlife, and it is known for its unique art style and dark humor.\n\n2. **Violet Evergarden** (13 episodes)\n\nViolet Evergarden is a drama anime series that follows the titular character, a former soldier who is now working as an Auto Memory Doll, a person who writes letters for others. The series explores themes of love, loss, and the power of words.\n\n3. **March Comes in Like a Lion** (22 episodes)\n\nMarch Comes in Like a Lion is a slice-of-life anime series that follows Rei Kiriyama, a young shogi player who is struggling to cope with the loss of his family. The series explores themes of loneliness, grief, and the importance of family and friends.\n\n4. **A Place Further Than the Un

In [14]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='codebasics_faqs.csv', source_column='prompt', encoding='latin-1')
data = loader.load()

In [ ]:
data

In [21]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores.faiss import FAISS

embeddings = HuggingFaceInstructEmbeddings()

vectordb = FAISS.from_documents(documents=data, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
retriever = vectordb.as_retriever()
retriever.get_relevant_documents("Is emi available?")

In [23]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and question, generate an answer based on the context.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
chain_type_kwargs = {"prompt": PROMPT}

In [24]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [25]:
chain("do you provide javascript course?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'do you provide javascript course?',
 'result': 'Yes, we do provide JavaScript courses. Our JavaScript courses are designed for beginners with no prior experience in coding. We start with the basics of JavaScript, such as variables, functions, and objects, and then move on to more advanced topics, such as loops, conditional statements, and arrays. We also cover topics such as DOM manipulation, event handling, and AJAX. Our JavaScript courses are taught by experienced and qualified instructors who are passionate about teaching. We offer a variety of JavaScript courses, including:\n\n* **JavaScript for Beginners:** This course is designed for students who have never coded before. We cover the basics of JavaScript, such as variables, functions, and objects.\n* **JavaScript Intermediate:** This course is designed for students who have some experience with JavaScript. We cover more advanced topics, such as loops, conditional statements, and arrays.\n* **JavaScript Advanced:** This

In [ ]:
chain("How to become data analyst?")

{'query': 'How to become data analyst?',
 'result': 'To become a data analyst, you will need to have a strong foundation in mathematics, statistics, and programming. You will also need to be able to work independently and be able to communicate your findings effectively.\n\nHere are some steps on how to become a data analyst:\n\n1. **Get a degree in data science or a related field.** This will give you the foundation in mathematics, statistics, and programming that you need to be a data analyst.\n2. **Gain experience in data analysis.** You can gain experience in data analysis by working as a data analyst intern, or by taking on data analysis projects on your own.\n3. **Develop your skills in data analysis.** There are many resources available to help you develop your skills in data analysis, such as online courses, books, and tutorials.\n4. **Network with other data analysts.** Attend industry events, join online communities, and connect with other data analysts on LinkedIn. This will

In [26]:
chain("Are you evil?")

{'query': 'Are you evil?',
 'result': 'I am not evil. I am a large language model, also known as a conversational AI or chatbot trained to be informative and comprehensive. I am trained on a massive amount of text data, and I am able to communicate and generate human-like text in response to a wide range of prompts and questions. For example, I can provide summaries of factual topics or create stories.\r\n\r\nI am still under development, and I am always learning new things. I am not perfect, and I sometimes make mistakes. However, I am always working to improve my skills and knowledge.\r\n\r\nI believe that I can be a valuable tool for learning and communication. I hope that you will find me helpful and informative.',
 'source_documents': [Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'s